## Tarefa 1: Treinar um modelo

O processo de criação de um modelo de machine learning (ML) começa com o processamento de dados. Após a conclusão do processamento de dados, você escolhe um algoritmo de ML para treinar seu modelo. O objetivo do treinamento do modelo é criar um modelo que você possa usar para fazer previsões com dados futuros. Seus dados processados devem conter um destino, mas seus dados futuros não contêm um destino (eles não estão rotulados). O algoritmo encontra padrões nos dados de treinamento que mapeiam os atributos dos dados de entrada para o destino. O algoritmo então gera um modelo de ML que captura esses padrões. Quando você tem um modelo, pode fazer previsões sobre novos dados que não contenham o valor de destino.

Por exemplo, caso queira treinar um modelo de ML para prever se um e-mail é spam ou não, forneça ao seu modelo dados de treinamento que contenham e-mails cujo destino você conhece (nesse caso, um rótulo que informa se um e-mail é spam ou não). Usando esses dados, o algoritmo cria um modelo que prevê se um e-mail é spam ou não. Você pode usar esse modelo para prever rótulos de e-mail futuros.

Nesta tarefa, você está prevendo se alguém tem menos de USD 50 mil ou não. Seu modelo está sendo treinado para se otimizar para poder prever se alguém tem menos de USD 50 mil com a maior precisão possível. O treinamento do modelo requer alguma configuração, incluindo o tipo de algoritmo que você deseja usar para treinar. Nesta tarefa, você usa o algoritmo XGBoost (eXtreme Gradient Boosting). Ao treinar um modelo, você também precisa configurar seus hiperparâmetros. Hiperparâmetros são parâmetros que controlam o processo de trabalho de treinamento. Eles podem ser ajustados para alterar diversas etapas do trabalho de treinamento. Selecionar o conjunto certo de hiperparâmetros é importante em termos de desempenho e acurácia do modelo. Depois de treinar o modelo, você vai avaliá-lo e visualizar os artefatos dele.

### Tarefa 1.1: Configurar o ambiente

Antes de iniciar o treinamento do modelo, instale todas as dependências necessárias.

In [ ]:
#Install matplotlib and restart kernel
%pip install matplotlib
%pip uninstall bokeh -y
%pip install bokeh==2.4.2
%reset -f

# Install dependencies
import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from sagemaker.debugger import Rule, rule_configs
from IPython.display import FileLink, FileLinks
from sagemaker import image_uris
from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime
from pathlib import Path

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')

Observação: caso seja exibido o erro na saída “ERROR: pip's dependency resolver does not currently take into account all the packages that are installed” (ERRO: o solucionador de dependência de pip não considera todos os pacotes instalados no momento), você pode ignorar o erro de dependência do pacote e prosseguir com as próximas etapas.

Depois, importe o conjunto de dados. Para este exercício, suponha que alguém tenha exportado os arquivos de dados do Amazon SageMaker Data Wrangler para um bucket do Amazon Simple Storage Service (Amazon S3). Você vai dividir o conjunto de dados nos seguintes conjuntos de dados: treinamento (70%), validação (20%) e teste (10%). Os conjuntos de dados de treinamento e validação são usados durante o treinamento. O conjunto de dados de teste é usado na avaliação do modelo após a implantação.

Para treinar usando o Amazon SageMaker, os conjuntos de dados devem estar no formato libSVM ou CSV. Este laboratório usa o formato CSV para o treinamento. 

Para visualizar os arquivos do conjunto de dados criados no laboratório anterior, siga estas etapas abaixo:

1. Navegue de volta para o Console de Gerenciamento da AWS.

1. Na parte superior do Console de Gerenciamento da AWS, na barra de pesquisa, pesquise e escolha `S3`.

1. Na lista de buckets, selecione o bucket do Amazon S3 que contém **labdatabucket** no nome.

1. Escolha a pasta **scripts**, a pasta **data**, a pasta**train** e, em seguida, o arquivo **adult_data_processed_train.csv**.

1. Baixe o arquivo**adult_data_processed_train.csv** para sua máquina local e revise-o usando um editor de sua escolha.

1. No console do Amazon S3, na parte superior da página, escolha **data** no link de navegação estrutural **labdatabucket-xxxx/scripts/data/train/**.

1. Escolha a pasta**validation** e, em seguida, escolha o arquivo **adult_data_processed_validation.csv**.

1. Baixe o arquivo **adult_data_processed_validation.csv** em sua máquina local e revise-o usando um editor de sua escolha.

Você visualizou os arquivos do conjunto de dados. Agora, configure os caminhos de treinamento e validação que seu trabalho de treinamento usa como entrada.

In [ ]:
# Import the datasets
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)
prefix = 'scripts/data'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Configure the training paths
train_path = f"s3://{bucket}/{prefix}/train/adult_data_processed_train.csv"
validation_path = f"s3://{bucket}/{prefix}/validation/adult_data_processed_validation.csv"

# Set up the TrainingInput objects
train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

# Print the training and validation paths
print(f'Training path: {train_path}')
print(f'Validation path: {validation_path}')

# Set the container, name, and tags
create_date = strftime("%m%d%H%M")
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')
run_name = 'lab-2-run-{}'.format(create_date)

### Tarefa 1.2: Configurar um objeto estimador

Um estimador é uma interface de alto nível para treinamento do SageMaker. Você cria um objeto estimador fornecendo os parâmetros necessários, como a função do AWS Identity and Access Management (AWS IAM), a contagem e o tipo de instâncias de computação e o caminho de saída do Amazon S3. Este laboratório usa o algoritmo integrado XGBoost para o estimador genérico SageMaker. O XGBoost é uma conhecida e eficiente implementação de código aberto do algoritmo baseado em árvores com aumento de gradiente. O aumento de gradiente é um algoritmo de aprendizado supervisionado que tenta prever com precisão uma variável de destino. Para isso, combina um grupo de estimativas de um conjunto de modelos mais simples e mais fracos. O algoritmo XGBoost tem um bom desempenho no tratamento de uma variedade de tipos de dados, relacionamentos, distribuições e uma variedade de hiperparâmetros que você pode ajustar. Você pode usar o XGBoost para problemas de regressão, classificação (binária e multiclasse) e classificação. Neste caso, você está usando o XGBoost para resolver um problema de classificação (se alguém ganha menos de USD 50 mil ou não).

Neste laboratório, você vai criar um estimador XGBoost usando a classe *sagemaker.estimator.Estimator*. No código de exemplo a seguir, o estimador XGBoost é denominado *xgb_model*. Para criar o estimador do SageMaker, especifique os seguintes parâmetros:

- **image_uri**: o URI da imagem do contêiner de treinamento. Neste exemplo, o URI do contêiner de treinamento de XGBoost do SageMaker é especificado usando *image_uris.retrieve*.
- **função**: o perfil do IAM que o SageMaker usa para executar tarefas em seu nome (por exemplo, ler resultados de treinamento, chamar artefatos de modelo do Amazon S3 e gravar resultados de treinamento no Amazon S3). 
- **instance_count e instance_type**: o tipo e o número de instâncias de computação de ML do o Amazon EC2 a serem usadas para treinamento de modelo. Neste laboratório, você vai usar uma única instância ml.m5.xlarge, que tem 4 CPUs, 16 GB de memória, armazenamento Amazon Elastic Block Store (Amazon EBS) e alto desempenho de rede.
- **output_path**: o caminho para o bucket do S3 onde o SageMaker armazena o artefato do modelo e os resultados do treinamento.
- **sagemaker_session**: o objeto de sessão que gerencia interações com operações de API do SageMaker e outros serviços da AWS usados pelo trabalho de treinamento.
- **regras**: uma lista de regras integradas do Amazon SageMaker Debugger. Neste exemplo, a regra create_xgboost_report() cria um relatório XGBoost que fornece informações sobre o progresso e os resultados do treinamento.

In [ ]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role, 
    instance_count = 1, 
    instance_type ='ml.m5.xlarge',
    output_path = output_path,
    sagemaker_session = sagemaker_session,
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report())
    ]
)

### Tarefa 1.3: Configurar hiperparâmetros

Os hiperparâmetros controlam diretamente a estrutura, a função e o desempenho do modelo. O ajuste de hiperparâmetros permite que os data scientists ajustem o desempenho do modelo para obter resultados ideais. Esse processo é uma parte essencial do machine learning, e a escolha de valores de hiperparâmetros apropriados é crucial para o sucesso.

Você pode definir hiperparâmetros para o algoritmo XGBoost chamando o método *set_hyperparameters* do estimador.

Consulte [XGBoost hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) (Hiperparâmetros do XGBoost) para saber mais sobre os hiperparâmetros do XGBoost.

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    verbosity = 0,
    objective = 'binary:logistic',
    num_round = 800
)

### Tarefa 1.4: Executar um trabalho de treinamento do SageMaker

Agora que configurou o objeto estimador e os hiperparâmetros, você pode começar a treinar o modelo. O método fit() inicia o script de treinamento. Para iniciar o treinamento do modelo, chame o método fit() do estimador com os conjuntos de dados de treinamento e de validação. Se você definir “wait=True”, o método fit() vai exibir logs de progresso e aguardar até que o treinamento seja concluído.

In [ ]:
xgb_model.fit(
    {
        "train": train_input,
        "validation": validation_input
    },
    wait=True
)

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i> **Saída esperada:** caso a configuração do estimador e do hiperparâmetro esteja correta e o trabalho de treinamento tenha sido iniciado corretamente, você deve ver a seguinte saída:

```plain
************************
**** EXEMPLO DE SAÍDA ****
************************

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-08-16-40-54-080
2024-10-08 16:40:55 Starting - Starting the training job...
2024-10-08 16:41:17 Starting - Preparing the instances for trainingCreateXgboostReport: InProgress
...
2024-10-08 16:41:57 Downloading - Downloading the training image......
2024-10-08 16:42:58 Training - Training image download completed. Training in progress.
......
2024-10-08 16:43:18 Uploading - Uploading generated training model
2024-10-08 16:43:18 Completed - Training job completed
Training seconds: 104
Billable seconds: 104
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Observação:** o treinamento leva de três a quatro minutos para ser executado.

### Tarefa 1.5: Avaliar um modelo

Após a conclusão do trabalho de treinamento, você pode baixar um relatório de treinamento XGBoost gerado pelo SageMaker Debugger. O relatório de treinamento XGBoost oferece informações sobre o progresso e os resultados do treinamento, como a função de perda em relação à iteração, a importância do recurso, a matriz de confusão, as curvas de acurácia e outros resultados estatísticos do treinamento. 

Para trabalhos de treinamento do SageMaker XGBoost, use a regra “CreateXgboostReport” do Debugger para obter um relatório de treinamento abrangente sobre o progresso e os resultados do treinamento.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Observação:** o relatório leva de cinco a dez minutos para ser finalizado. No bloco de código a seguir, é verificado se o relatório final está presente. Se ainda não existir um relatório, o sistema vai esperar 30 segundos, copiar todos os novos arquivos da pasta de saída para o caminho local e verificar novamente até que o relatório fique disponível.

In [ ]:
%%capture
the_file = Path("CreateXgboostReport/xgboost_report.ipynb")
notexists = True
while notexists:
    if the_file.is_file():
        notexists = False
    else:
        time.sleep(30)
        rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.job_name + "/rule-output"
        ! aws s3 ls {rule_output_path} --recursive
        ! aws s3 cp {rule_output_path} ./ --recursive

O link na saída da próxima célula abre uma nova guia no SageMaker Studio.

In [ ]:
display("Click link below to view the XGBoost Training notebook", FileLink("CreateXgboostReport/xgboost_report.ipynb"))

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Observação:** depois da execução deste código, você deve ver a seguinte saída: **“Click link below to view the XGBoost Training notebook”** (Clique no link abaixo para visualizar o bloco de anotações do treinamento XGBoost) <span style="ssb_sm_blue">CreateXgboostReport/xgboost_report.ipynb</span>**

1. Para abrir o bloco de anotações em uma nova guia, selecione o link.

1. Quando o notebook for aberto, na janela pop-up **Select Kernel** (Selecionar kernel), no parâmetro **Select kernel for:** (Selecionar kernel para:), escolha **Python 3 (ipykernel)**.

1. Clique em <span style="background-color:#3873CA; font-weight:bold; font-size:80%; color:white; position:relative; top:-1px; border-radius:1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; margin-right:5px; white-space:nowrap">Selecionar</span>.

Na parte superior da guia **xgboost_report.ipynb**, escolha o botão <i aria-hidden="true" class="fas fa-forward"></i> **Restart the kernel and run all cells** (Reiniciar o kernel e executar todas as células). Quando for perguntado **Restart Kernel?** (Reiniciar Kernel?), selecione **Restart** (Reiniciar)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Observação:** leva aproximadamente de dois a três minutos para executar todas as células.

Após todas as células serem executadas, role para baixo até chegar a **Confusion Matrix** (Matriz de confusão). A matriz de confusão ilustra em uma tabela o número de previsões corretas e incorretas para cada classe, comparando a classe prevista de uma observação e sua classe verdadeira. Ao acessar o diagrama, você vê valores **verdadeiro positivo (TP)**, **verdadeiro negativo (TN)**, **falso positivo (FP)** e **falso negativo (FN)**.

- **Verdadeiro positivo:** se a classificação real for positiva e a classificação prevista for positiva (1,1), isso é chamado de resultado **verdadeiro positivo (TP)**, porque a amostra positiva foi identificada corretamente pelo classificador. 
- **Falso negativo:** se a classificação real for positiva e a classificação prevista for negativa (1,0), isso é chamado de resultado de **falso negativo (FN)**, porque a amostra positiva é identificada incorretamente pelo classificador como sendo negativa. 
- **Falso positivo:** se a classificação real for negativa e a classificação prevista for positiva (0,1), isso é chamado de resultado de **falso positivo (FP)**, porque a amostra negativa é identificada incorretamente pelo classificador como sendo positiva. 
- **Verdadeiro negativo**: se a classificação real for negativa e a classificação prevista for negativa (0,0), isso é chamado de resultado **verdadeiro negativo (TN)**, porque a amostra negativa é identificada corretamente pelo classificador.

Em seguida, role para baixo até **Avaliação da matriz de confusão** e dê uma olhada mais de perto no **Relatório de classificação** para entender o resumo da precisão, o recall e a pontuação F1 de cada classe.

- **Precisão**: mede a fração de positivos reais que foram previstos como positivos entre todos os previstos como positivos. O intervalo é de 0 a 1 e um valor maior indica melhor acurácia. A precisão expressa a proporção de pontos de dados que seu modelo diz serem relevantes e que eram realmente relevantes. A precisão é uma boa medida a considerar, especialmente quando os custos de PF são elevados.
- **Recall/sensibilidade/taxa de verdadeiro positivo (TPR)**: mede a fração de positivos reais que foram previstos como positivos. O intervalo também é de 0 a 1, e um valor maior indica uma melhor acurácia preditiva. Isso também é conhecido como Recall/sensibilidade. Essa medida expressa a capacidade de encontrar todas as instâncias relevantes em um conjunto de dados.
- **Pontuação F1**: demonstra sua métrica-alvo, que é a média harmônica de precisão e recall. A F1 leva em consideração os FP e os FN para dar o mesmo peso à precisão e ao recall.

Você está tentando prever se as pessoas ganham menos de USD 50 mil para poder promover serviços de assistência governamental a cidadãos qualificados. Nesse caso, a pontuação F1 é uma boa medida a ser usada, porque leva em consideração os FP (pessoas que ganham mais de USD 50 mil e que foram rotuladas como ganhando menos de USD 50 mil) e os FN (pessoas que ganham menos de USD 50 mil e que foram rotuladas como ganhando mais de USD 50 mil) em conta. Você quer ter certeza de que sua precisão e recall são altos, e a pontuação F1 leva ambas as medidas em consideração. No próximo laboratório, você vai otimizar o modelo ajustando os hiperparâmetros para ver se consegue uma pontuação F1 mais alta.

Quais são a **Precisão**, **Recall**, **Pontuação F1** e **Acurácia geral** para este modelo?

<i aria-hidden="true" class="far fa-comment" style="color:#008296"></i> **Considere:** verifique os outros gráficos que estão inclusos no bloco de anotações. Que tipo de informação você vê? O que pode ser útil para você enquanto treina seus próprios modelos?

### Tarefa 1.6: Visualizar os artefatos do modelo

O SageMaker armazena o artefato do modelo em seu bucket do S3. Para encontrar a localização do artefato do modelo, siga estas etapas:

1. Navegue de volta para o Console de Gerenciamento da AWS.

1. Na parte superior do Console de Gerenciamento da AWS, na barra de pesquisa, pesquise e escolha `S3`.

1. Na lista de buckets, selecione o bucket do Amazon S3 que contém **labdatabucket** no nome.

1. Navegue até a subpasta **scripts/data/output/sagemaker-xgboost-.../output**. 

Você verá o artefato do modelo **model.tar.gz** na subpasta. Esse é o modelo que você criou com seu SageMaker Estimator chamando o método fit().

Você visualizou os artefatos do modelo, incluindo o arquivo model.tar.gz. 

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

– Feche este arquivo do caderno.
– Retorne à sessão de laboratório e prossiga para a tarefa 2.